In [1]:
# -*- coding: utf-8 -*-
import csv
import tqdm
import copy
import click
import logging
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset

from string import digits

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torch.nn.functional as F

seed = 42
epochs = 100
batch_size = 80
learning_rate = 1e-3
context_frames = 10
sequence_length = 20
lookback = sequence_length

valid_train_split = 0.85  # precentage of train data from total
test_train_split = 0.92  # precentage of train data from total

image_height = 32
image_width  = 32

torch.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")#  use gpu if available
################################# CHANGE THIS!!!!  #################################
model_path = "/home/user/Robotics/slip_detection_model/slip_detection_model/manual_data_models/models/CorlPaperFinal/conv_model/"
data_dir = "/home/user/Robotics/Data_sets/slip_detection/will_dataset/will_data_collection/formated_image_data/"
################################# CHANGE THIS!!!!  #################################

In [6]:
class BatchGenerator:
    def __init__(self):
        self.data_dir = data_dir
        data_map = []
        with open(data_dir + 'map.csv', 'r') as f:  # rb
            reader = csv.reader(f)
            for row in reader:
                data_map.append(row)

        if len(data_map) <= 1: # empty or only header
            print("No file map found")
            exit()

        self.data_map = data_map

    def load_full_data(self):
        dataset_train = FullDataSet(self.data_dir, self.data_map, type_="train")
        dataset_valid = FullDataSet(self.data_dir, self.data_map, type_="valid")
        dataset_test = FullDataSet(self.data_dir, self.data_map, type_="test")
        transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
        train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
        valid_loader = torch.utils.data.DataLoader(dataset_valid, batch_size=batch_size, shuffle=True)
        test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size, shuffle=False)
        return train_loader, valid_loader, test_loader

class FullDataSet():
    def __init__(self, data_dir, data_map, type_="train"):
        if type_ == "train":
            self.samples = data_map[1:int(len(data_map)*test_train_split)]
        elif type_ == "valid":
            self.samples = data_map[int(len(data_map)*(valid_train_split)):int(len(data_map)*test_train_split)]
        elif type_ == "test":
            self.samples = data_map[int(len(data_map)*test_train_split):-1]
        data_map = None

    def __len__(self):
        return len(self.samples)

    def __getitem__(self,idx):
        # neet to scale between 0 and 1:
        value = self.samples[idx]
        robot = np.load(data_dir + value[0])
        xela1image = [np.load(data_dir + value[i]).squeeze() for i in range(1, sequence_length+1)]
        experiment = np.load(data_dir + value[-2])
        time_step  = np.load(data_dir + value[-1])
        return([robot.astype(np.float32),
                np.array(xela1image).astype(np.float32),
                experiment,
                time_step])

In [7]:
import torch
from torch import nn
import torch.nn.functional as f
from torch.autograd import Variable

class ConvLSTMCell(nn.Module):
    def __init__(self, input_dim, hidden_dim, kernel_size, bias):
        super(ConvLSTMCell, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.padding = kernel_size[0] // 2, kernel_size[1] // 2
        self.bias = bias
        self.conv = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim, out_channels=4 * self.hidden_dim,
                              kernel_size=self.kernel_size, padding=self.padding, bias=self.bias)

    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state
        combined = torch.cat([input_tensor, h_cur], dim=1)  # concatenate along channel axis
        combined_conv = self.conv(combined)
        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1)
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)
        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)
        return h_next, c_next

    def init_hidden(self, batch_size, image_size):
        height, width = image_size
        return (torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device).to(device),
                torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device).to(device))

# initial test:
class FullModel(nn.Module):
    def __init__(self):
        super(FullModel, self).__init__()
        self.convlstm1 = ConvLSTMCell(input_dim=3, hidden_dim=12, kernel_size=(3, 3), bias=True).cuda()
        self.convlstm2 = ConvLSTMCell(input_dim=24, hidden_dim=24, kernel_size=(3, 3), bias=True).cuda()
        self.conv1 = nn.Conv2d(in_channels=24, out_channels=3, kernel_size=5, stride=1, padding=2).cuda()

    def forward(self, tactiles, actions):
        self.batch_size = actions.shape[1]
        state = actions[0]
        state.to(device)
        batch_size__ = tactiles.shape[1]
        hidden_1, cell_1 = self.convlstm1.init_hidden(batch_size=self.batch_size, image_size=(32, 32))
        hidden_2, cell_2 = self.convlstm2.init_hidden(batch_size=self.batch_size, image_size=(32, 32))
        outputs = []
        for index, (sample_tactile, sample_action) in enumerate(zip(tactiles.squeeze(), actions.squeeze())):
            sample_tactile.to(device)
            sample_action.to(device)
            # 2. Run through lstm:
            if index > context_frames-1:
                hidden_1, cell_1 = self.convlstm1(input_tensor=output, cur_state=[hidden_1, cell_1])
                state_action = torch.cat((state, sample_action), 1)
                robot_and_tactile = torch.cat((torch.cat(32*[torch.cat(32*[state_action.unsqueeze(2)], axis = 2).unsqueeze(3)], axis = 3), hidden_1.squeeze()), 1)
                hidden_2, cell_2 = self.convlstm2(input_tensor=robot_and_tactile, cur_state=[hidden_2, cell_2])
                output = self.conv1(hidden_2)
                outputs.append(output)
            else:
                hidden_1, cell_1 = self.convlstm1(input_tensor=sample_tactile, cur_state=[hidden_1, cell_1])
                state_action = torch.cat((state, sample_action), 1)
                robot_and_tactile = torch.cat((torch.cat(32*[torch.cat(32*[state_action.unsqueeze(2)], axis = 2).unsqueeze(3)], axis = 3), hidden_1.squeeze()), 1)
                hidden_2, cell_2 = self.convlstm2(input_tensor=robot_and_tactile, cur_state=[hidden_2, cell_2])
                output = self.conv1(hidden_2)
        return torch.stack(outputs)

In [9]:
class ModelTrainer:
    def __init__(self):
        self.train_full_loader, self.valid_full_loader, self.test_full_loader = BG.load_full_data()
        self.full_model = FullModel()
        self.criterion = nn.L1Loss()
        self.criterion1 = nn.L1Loss()
        self.optimizer = optim.Adam(self.full_model.parameters(), lr=learning_rate)

    def train_full_model(self):
        plot_training_loss = []
        plot_validation_loss = []
        previous_val_mean_loss = 100.0
        best_val_loss = 100.0
        early_stop_clock = 0
        progress_bar = tqdm.tqdm(range(0, epochs), total=(epochs*len(self.train_full_loader)))
        mean_test = 0
        for epoch in progress_bar:
            loss = 0.0
            losses = 0.0
            for index, batch_features in enumerate(self.train_full_loader):
                action = batch_features[0].squeeze().permute(1,0,2).to(device)
                tactile = batch_features[1].permute(1,0,4,3,2).to(device)
                tactile_predictions = self.full_model.forward(tactiles=tactile, actions=action) # Step 3. Run our forward pass.
                self.optimizer.zero_grad()
                loss = self.criterion(tactile_predictions, tactile[context_frames:])
                loss.backward()
                self.optimizer.step()

                losses += loss.item()
                if index:
                    mean = losses / index
                else:
                    mean = 0
                progress_bar.set_description("epoch: {}, ".format(epoch) + "loss: {:.4f}, ".format(float(loss.item())) + "mean loss: {:.4f}, ".format(mean))
                progress_bar.update()
            plot_training_loss.append(mean)

            val_losses = 0.0
            val_loss = 0.0
            with torch.no_grad():
                for index__, batch_features in enumerate(self.valid_full_loader):
                    action = batch_features[0].squeeze().permute(1,0,2).to(device)
                    tactile = batch_features[1].permute(1,0,4,3,2).to(device)
                    tactile_predictions = self.full_model.forward(tactiles=tactile, actions=action) # Step 3. Run our forward pass.                    
                    self.optimizer.zero_grad()
                    val_loss = self.criterion(tactile_predictions.to(device), tactile[context_frames:])
                    val_losses += val_loss.item()

            print("Validation mean loss: {:.4f}, ".format(val_losses / index__))
            plot_validation_loss.append(val_losses / index__)
            if previous_val_mean_loss < val_losses / index__:
                early_stop_clock +=1
                previous_val_mean_loss = val_losses / index__ 
                if early_stop_clock == 3:
                    print("Early stopping")
                    break
            else:
                if best_val_loss > val_losses / index__:
                    print("saving model")
                    torch.save(self.full_model, model_path + "conv_model")
                    self.strongest_model = copy.deepcopy(self.full_model)
                    best_val_loss = val_losses / index__
                early_stop_clock = 0
                previous_val_mean_loss = val_losses / index__ 
            plt.plot(plot_training_loss, c="r", label="train loss MAE")
            plt.plot(plot_validation_loss, c='b', label="val loss MAE")
            plt.legend(loc="upper right")
            plt.savefig(model_path + '/trining_plot_10steppred_new_data.png', dpi=300)

In [ ]:
## create data generator, train model and save model.
BG = BatchGenerator()
MT = ModelTrainer()
MT.train_full_model()
print("finished training")
torch.save(MT.strongest_model, model_path + "model_t1")
model = torch.load(model_path + "model_t1")
model.eval()
print("saved the model")

from twilio.rest import Client
account_sid = "ACbb524ad1deed7bd1d53d2be7dd189de6"
auth_token = "9a0a01cddac5b0c827eb89a534fa897e"
client = Client(account_sid, auth_token)
FROM_NUM        = "+12562861398"
VERIFIED_NUMBER = "+447722355970"
message = client.messages.create(body='TESTING', from_=[FROM_NUM], to=[VERIFIED_NUMBER])
print(message.sid)

epoch: 0, loss: 0.0191, mean loss: 0.0277, :   1%|          | 2249/224800 [27:43<1500:54:39, 24.28s/it]

Validation mean loss: 0.0194, 
saving model


epoch: 1, loss: 0.0125, mean loss: 0.0157, :   2%|▏         | 4497/224800 [53:42<40:01:31,  1.53it/s]  

Validation mean loss: 0.0142, 
saving model


epoch: 2, loss: 0.0135, mean loss: 0.0139, :   3%|▎         | 6745/224800 [1:21:01<39:02:20,  1.55it/s]

...

epoch: 37, loss: 0.0067, mean loss: 0.0079, :  38%|███▊      | 85425/224800 [17:09:36<24:51:23,  1.56it/s] 

Validation mean loss: 0.0077, 
saving model


epoch: 38, loss: 0.0090, mean loss: 0.0078, :  39%|███▉      | 87673/224800 [17:36:29<26:40:15,  1.43it/s] 

Validation mean loss: 0.0079, 


epoch: 39, loss: 0.0072, mean loss: 0.0078, :  39%|███▉      | 87839/224800 [17:39:42<26:05:15,  1.46it/s] 

In [ ]:
model = torch.load(model_path + "conv_model")
model.eval()
BG = BatchGenerator()
train_full_loader, valid_full_loader, test_full_loader = BG.load_full_data()

In [ ]:
# creates a sequnce of tactile readings for test data with sample of 
import cv2
criterion1 = nn.L1Loss()
criterion2 = nn.MSELoss()

experiments = []
tactile_predictions = []
tactile_groundtruth = []
experiment_time_steps = []

with torch.no_grad():
    for index__, batch_features in enumerate(test_full_loader):
        # 2. Reshape data and send to device:                    
        action = batch_features[0].squeeze().permute(1,0,2).to(device)
        tactile = batch_features[1].permute(1,0,4,3,2).to(device)

        tp = model.forward(tactiles=tactile, actions=action)
        experiments.append(batch_features[2])
        experiment_time_steps.append([batch_features[2], batch_features[3]])
        
        # test for 0 and gt + 10:
        image_pred_batch = []
        image_gt_batch = []
        for image_pred, image_gt in zip(tp[-1], tactile[context_frames-1:][0]):
            image_pred_batch.append(cv2.resize(image_pred.permute(1,2,0).cpu().detach().numpy(), dsize=(4,4), interpolation=cv2.INTER_CUBIC).flatten())
            image_gt_batch.append(cv2.resize(image_gt.permute(1,2,0).cpu().detach().numpy(), dsize=(4,4), interpolation=cv2.INTER_CUBIC).flatten())
        tactile_predictions.append(np.array(image_pred_batch))
        tactile_groundtruth.append(np.array(image_gt_batch))

In [ ]:
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
experiment_to_test = 210
prediction_data_to_plot = []
gt_data_to_plot = []
time_stepsto_plot = []
for index, (pred_batch, gt_batch) in enumerate(zip(tactile_predictions, tactile_groundtruth)):
    for batch_step, (prediction_sample, gt_sample)  in enumerate(zip(pred_batch, gt_batch)):
        experiment              = experiment_time_steps[index][0][batch_step][0].numpy()
        time_step_in_experiment = experiment_time_steps[index][1][batch_step][0].numpy()
        if experiment == experiment_to_test:
            prediction_data_to_plot.append(prediction_sample)
            gt_data_to_plot.append(gt_sample)
            time_stepsto_plot.append(time_step_in_experiment)


plot_range_start = 50 # 0
plot_range_stop = 100 # -1
for tactile in range(48):
        fig, ax1 = plt.subplots()
        ax1.set_xlabel('time step')
        ax1.set_ylabel('tactile reading')
        ax1.plot([value for value in time_stepsto_plot[plot_range_start:plot_range_stop]], np.array(prediction_data_to_plot)[plot_range_start:plot_range_stop,tactile], alpha=0.5, c="g", label="t10")
        ax1.plot([value for value in time_stepsto_plot[plot_range_start:plot_range_stop]], np.array(gt_data_to_plot)[plot_range_start:plot_range_stop,tactile], alpha=0.5, c="r", label="gt")
        ax1.tick_params(axis='y')
        fig.tight_layout()  # otherwise the right y-label is slightly clipped
        fig.subplots_adjust(top=0.90)
        ax1.legend(loc="upper right")

        ax1.xaxis.set_minor_locator(AutoMinorLocator())
        ax1.tick_params(which='minor', length=1, color='k')
        ax1.grid(which='minor')
        ax1.grid(which='major')
#         ax1.set_ylim((0, 1.0))

        plt.title("Inline Simple_LSTM tactile " + str(index))
#         plt.savefig(model_path + '/' + str(experiment_to_test) + '/zeros_bigger_pconcat_double_data_model_sample_test_with_loss_' + str(index) + '.png', dpi=300)
        plt.show()

    

In [ ]:
# from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
# # calculate tactile values for full sample:
# time_step_to_test_t1 = 0    # [batch_set, prediction frames(t1->tx)(6), batch_size, features(48)]
# time_step_to_test_t5 = 4
# time_step_to_test_t9 = 9
# predicted_data_t1 = []
# predicted_data_t5 = []
# predicted_data_t9 = []
# groundtruth_data = []
# index_test = 0
# experiment_to_test = 210
# for index, batch_set in enumerate(tactile_predictions):
#     for batch in range(0, len(batch_set[0])):
#         experiment = experiment_time_steps[index][0][batch][0]
#         if experiment == experiment_to_test:
#             if not index_test == 0 : ## for use in single prediction plotting 
#                 single_prediction = batch_set
#                 single_ground_truth = tactile_groundtruth[index]
#             prediction_values = batch_set[time_step_to_test_t1][batch]
#             predicted_data_t1.append(prediction_values)
#             prediction_values = batch_set[time_step_to_test_t5][batch]
#             predicted_data_t5.append(prediction_values)
#             prediction_values = batch_set[time_step_to_test_t9][batch]
#             predicted_data_t9.append(prediction_values)
#             gt_values = tactile_groundtruth[index][time_step_to_test_t1][batch]
#             groundtruth_data.append(gt_values)
#             index_test = 1

# test data
index = 0
experiment_to_test = 210
titles = ["sheerx", "sheery", "normal"]
for j in range(3):
    for i in range(16):
        groundtruth_taxle = []
        predicted_taxel = []
        predicted_taxel_t1 = []
        predicted_taxel_t5 = []
        predicted_taxel_t9 = []
        # good = 140, 145 (lifting up the )
        for k in range(len(predicted_data_t1)): #320, 420):# len(predicted_data_t1)):#310, 325):#len(predicted_data_t1)):  # add in length of context data
            predicted_taxel_t1.append(predicted_data_t1[k][j+i])
            predicted_taxel_t5.append(predicted_data_t5[k][j+i])
            predicted_taxel_t9.append(predicted_data_t9[k][j+i])
            groundtruth_taxle.append(groundtruth_data[k][j+i])

        index += 1
        fig, ax1 = plt.subplots()
        ax1.set_xlabel('time step')
        ax1.set_ylabel('tactile reading')
        ax1.plot([i for i in predicted_taxel_t1], alpha=0.5, c="b", label="t1")
        ax1.plot([i for i in predicted_taxel_t5], alpha=0.5, c="k", label="t5")
        ax1.plot([i for i in predicted_taxel_t9], alpha=0.5, c="g", label="t10")
        ax1.plot(groundtruth_taxle, alpha=0.5, c="r", label="gt")
        ax1.tick_params(axis='y')
        fig.tight_layout()  # otherwise the right y-label is slightly clipped
        fig.subplots_adjust(top=0.90)
        ax1.legend(loc="upper right")
        
        ax1.xaxis.set_minor_locator(AutoMinorLocator())
        ax1.tick_params(which='minor', length=1, color='k')
        ax1.grid(which='minor')
        ax1.grid(which='major')
        ax1.set_ylim((0, 1.0))

        plt.title("Inline Simple_LSTM tactile " + str(index))
#         plt.savefig(model_path + '/' + str(experiment_to_test) + '/zeros_bigger_pconcat_double_data_model_sample_test_with_loss_' + str(index) + '.png', dpi=300)
        plt.show()

#         fig, ax1 = plt.subplots()
#         ax1.set_xlabel('time step')
#         ax1.set_ylabel('tactile reading')
#         ax1.plot([i for i in predicted_taxel_t1], alpha=0.5, c="b", label="t1")
#         ax1.plot(groundtruth_taxle, alpha=0.5, c="r", label="gt")
#         ax1.tick_params(axis='y')
#         fig.tight_layout()  # otherwise the right y-label is slightly clipped
#         fig.subplots_adjust(top=0.90)
#         ax1.legend(loc="upper right")
#         plt.title("Inline Simple_LSTM tactile " + str(index))
# #         plt.savefig(model_path + '/' + str(experiment_to_test) + '/zeros_bigger_pconcat_double_data_model_T0sample_test_with_loss_' + str(index) + '.png', dpi=300)
#         plt.show()
        
#         fig, ax1 = plt.subplots()
#         ax1.set_xlabel('time step')
#         ax1.set_ylabel('tactile reading')
#         ax1.plot([i for i in predicted_taxel_t5], alpha=0.5, c="b", label="t5")
#         ax1.plot(groundtruth_taxle, alpha=0.5, c="r", label="gt")
#         ax1.tick_params(axis='y')
#         fig.tight_layout()  # otherwise the right y-label is slightly clipped
#         fig.subplots_adjust(top=0.90)
#         ax1.legend(loc="upper right")
#         plt.title("Inline Simple_LSTM tactile " + str(index))
# #         plt.savefig(model_path + '/' + str(experiment_to_test) + '/zeros_bigger_pconcat_double_data_model_T5sample_test_with_loss_' + str(index) + '.png', dpi=300)
#         plt.show()
            
#         fig, ax1 = plt.subplots()
#         ax1.set_xlabel('time step')
#         ax1.set_ylabel('tactile reading')
#         ax1.plot([i for i in predicted_taxel_t9], alpha=0.5, c="b", label="t10")
#         ax1.plot(groundtruth_taxle, alpha=0.5, c="r", label="gt")
#         ax1.tick_params(axis='y')
#         fig.tight_layout()  # otherwise the right y-label is slightly clipped
#         fig.subplots_adjust(top=0.90)
#         ax1.legend(loc="upper right")
            
#         ax1.xaxis.set_minor_locator(AutoMinorLocator())
#         ax1.tick_params(which='minor', length=1, color='k')
#         ax1.grid(which='minor')
#         ax1.grid(which='major')
        
#         plt.title("Inline Simple_LSTM tactile " + str(index))
# #         plt.savefig(model_path + '/' + str(experiment_to_test) + '/zeros_bigger_pconcat_double_data_pca_model_T9sample_test_with_loss_' + str(index) + '.png', dpi=300)
#         plt.show()

In [ ]:
prediction = single_prediction[:,49,:]
ground_truth = single_ground_truth[:,49,:]

print(prediction.shape)
print(ground_truth.shape)

for i in range(48):
    fig, ax1 = plt.subplots()
    ax1.set_xlabel('future time step')
    ax1.set_ylabel('tactile reading')
    ax1.plot(prediction[:,i], alpha=0.5, c="b", label="single prediction")
    ax1.plot(ground_truth[:,i], alpha=0.5, c="r", label="gt")
    ax1.tick_params(axis='y')
    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    fig.subplots_adjust(top=0.90)
    ax1.legend(loc="upper right")
    ax1.xaxis.set_major_locator(MultipleLocator(1))
    ax1.grid(which='major')
    ax1.set_ylim((0, 1.0))
#     plt.savefig(model_path + '/' + str(experiment_to_test) + '/pconcat_double_data_model_single_prediction_sample_' + str(i) + '.png', dpi=300)
    plt.title("Simple_LSTM for single prediction sequence (0-10), tactile " + str(i))
    plt.show()

In [ ]:
# test model on the full test sample:
# test model on the full test sample:
model = torch.load(model_path + "conv1_model")
model.eval()
BG = BatchGenerator()
train_full_loader, valid_full_loader, test_full_loader = BG.load_full_data()

criterion1 = nn.L1Loss()
criterion2 = nn.MSELoss()

test_lossesMAE_x = 0.0
test_lossesMSE_x = 0.0
test_lossesMAE_y = 0.0
test_lossesMSE_y = 0.0
test_lossesMAE_z = 0.0
test_lossesMSE_z = 0.0

test_lossesMAE_t1 = 0.0
test_lossesMSE_t1 = 0.0
test_lossesMAE_t5 = 0.0
test_lossesMSE_t5 = 0.0
test_lossesMAE_t10 = 0.0
test_lossesMSE_t10 = 0.0

test_lossesMAE_x_ts1 = 0.0
test_lossesMSE_x_ts1 = 0.0
test_lossesMAE_y_ts1 = 0.0
test_lossesMSE_y_ts1 = 0.0
test_lossesMAE_z_ts1 = 0.0
test_lossesMSE_z_ts1 = 0.0

test_lossesMAE_x_ts5 = 0.0
test_lossesMSE_x_ts5 = 0.0
test_lossesMAE_y_ts5 = 0.0
test_lossesMSE_y_ts5 = 0.0
test_lossesMAE_z_ts5 = 0.0
test_lossesMSE_z_ts5 = 0.0

test_lossesMAE_x_ts10 = 0.0
test_lossesMSE_x_ts10 = 0.0
test_lossesMAE_y_ts10 = 0.0
test_lossesMSE_y_ts10 = 0.0
test_lossesMAE_z_ts10 = 0.0
test_lossesMSE_z_ts10 = 0.0

experiments = []
tactile_predictions = []
tactile_groundtruth = []
experiment_time_steps = []

test_lossesMAE = 0.0
test_lossesMSE = 0.0
with torch.no_grad():
    for index__, batch_features in enumerate(test_full_loader):
        # 2. Reshape data and send to device:                    
        action = batch_features[0].squeeze().permute(1,0,2).to(device)
        tactile = batch_features[1].permute(1,0,4,3,2).to(device)

        tp = model.forward(tactiles=tactile, actions=action)
        experiments.append(batch_features[2])
        experiment_time_steps.append([batch_features[2], batch_features[3]])
        tactile_predictions.append(tp)  # Step 3. Run our forward pass.
        tactile_groundtruth.append(tactile[context_frames-1:])
#         # calculate losses for specific timesteps
#         test_lossMAE_t1 = criterion1(tp[0,:,:].to(device), tactile[context_frames:][0,:,:])
#         test_lossesMAE_t1 += test_lossMAE_t1.item() 
#         test_lossMSE_t1 = criterion2(tp[0,:,:].to(device), tactile[context_frames:][0,:,:])
#         test_lossesMSE_t1 += test_lossMSE_t1.item() 
#         test_lossMAE_t5 = criterion1(tp[4,:,:].to(device), tactile[context_frames:][4,:,:])
#         test_lossesMAE_t5 += test_lossMAE_t5.item() 
#         test_lossMSE_t5 = criterion2(tp[4,:,:].to(device), tactile[context_frames:][4,:,:])
#         test_lossesMSE_t5 += test_lossMSE_t5.item() 
#         test_lossMAE_t10 = criterion1(tp[9,:,:].to(device), tactile[context_frames:][9,:,:])
#         test_lossesMAE_t10 += test_lossMAE_t10.item() 
#         test_lossMSE_t10 = criterion2(tp[9,:,:].to(device), tactile[context_frames:][9,:,:])
#         test_lossesMSE_t10 += test_lossMSE_t10.item() 
        
#         # calculate losses for specific forces
#         test_lossMAE_x = criterion1(tp[:,:,:16].to(device), tactile[context_frames:][:,:,:16])
#         test_lossesMAE_x += test_lossMAE_x.item() 
#         test_lossMSE_x = criterion2(tp[:,:,:16].to(device), tactile[context_frames:][:,:,:16])
#         test_lossesMSE_x += test_lossMSE_x.item() 
#         test_lossMAE_y = criterion1(tp[:,:,17:32].to(device), tactile[context_frames:][:,:,17:32])
#         test_lossesMAE_y += test_lossMAE_y.item() 
#         test_lossMSE_y = criterion2(tp[:,:,17:32].to(device), tactile[context_frames:][:,:,17:32])
#         test_lossesMSE_y += test_lossMSE_y.item() 
#         test_lossMAE_z = criterion1(tp[:,:,33:48].to(device), tactile[context_frames:][:,:,33:48])
#         test_lossesMAE_z += test_lossMAE_z.item() 
#         test_lossMSE_z = criterion2(tp[:,:,33:48].to(device), tactile[context_frames:][:,:,33:48])
#         test_lossesMSE_z += test_lossMSE_z.item() 

#         # calculate losses for specific timesteps and forces 
#         test_lossMAE_x_ts1 = criterion1(tp[0,:,:16].to(device), tactile[context_frames:][0,:,:16])
#         test_lossesMAE_x_ts1 += test_lossMAE_x_ts1.item() 
#         test_lossMSE_x_ts1 = criterion2(tp[0,:,:16].to(device), tactile[context_frames:][0,:,:16])
#         test_lossesMSE_x_ts1 += test_lossMSE_x_ts1.item() 
#         test_lossMAE_y_ts1 = criterion1(tp[0,:,17:32].to(device), tactile[context_frames:][0,:,17:32])
#         test_lossesMAE_y_ts1 += test_lossMAE_y_ts1.item() 
#         test_lossMSE_y_ts1 = criterion2(tp[0,:,17:32].to(device), tactile[context_frames:][0,:,17:32])
#         test_lossesMSE_y_ts1 += test_lossMSE_y_ts1.item() 
#         test_lossMAE_z_ts1 = criterion1(tp[0,:,33:48].to(device), tactile[context_frames:][0,:,33:48])
#         test_lossesMAE_z_ts1 += test_lossMAE_z_ts1.item() 
#         test_lossMSE_z_ts1 = criterion2(tp[0,:,33:48].to(device), tactile[context_frames:][0,:,33:48])
#         test_lossesMSE_z_ts1 += test_lossMSE_z_ts1.item() 
 
#         test_lossMAE_x_ts5 = criterion1(tp[4,:,:16].to(device), tactile[context_frames:][4,:,:16])
#         test_lossesMAE_x_ts5 += test_lossMAE_x_ts5.item() 
#         test_lossMSE_x_ts5 = criterion2(tp[4,:,:16].to(device), tactile[context_frames:][4,:,:16])
#         test_lossesMSE_x_ts5 += test_lossMSE_x_ts5.item() 
#         test_lossMAE_y_ts5 = criterion1(tp[4,:,17:32].to(device), tactile[context_frames:][4,:,17:32])
#         test_lossesMAE_y_ts5 += test_lossMAE_y_ts5.item() 
#         test_lossMSE_y_ts5 = criterion2(tp[4,:,17:32].to(device), tactile[context_frames:][4,:,17:32])
#         test_lossesMSE_y_ts5 += test_lossMSE_y_ts5.item() 
#         test_lossMAE_z_ts5 = criterion1(tp[4,:,33:48].to(device), tactile[context_frames:][4,:,33:48])
#         test_lossesMAE_z_ts5 += test_lossMAE_z_ts5.item() 
#         test_lossMSE_z_ts5 = criterion2(tp[4,:,33:48].to(device), tactile[context_frames:][4,:,33:48])
#         test_lossesMSE_z_ts5 += test_lossMSE_z_ts5.item() 

#         test_lossMAE_x_ts10 = criterion1(tp[9,:,:16].to(device), tactile[context_frames:][9,:,:16])
#         test_lossesMAE_x_ts10 += test_lossMAE_x_ts10.item() 
#         test_lossMSE_x_ts10 = criterion2(tp[9,:,:16].to(device), tactile[context_frames:][9,:,:16])
#         test_lossesMSE_x_ts10 += test_lossMSE_x_ts10.item() 
#         test_lossMAE_y_ts10 = criterion1(tp[9,:,17:32].to(device), tactile[context_frames:][9,:,17:32])
#         test_lossesMAE_y_ts10 += test_lossMAE_y_ts10.item() 
#         test_lossMSE_y_ts10 = criterion2(tp[9,:,17:32].to(device), tactile[context_frames:][9,:,17:32])
#         test_lossesMSE_y_ts10 += test_lossMSE_y_ts10.item() 
#         test_lossMAE_z_ts10 = criterion1(tp[9,:,33:48].to(device), tactile[context_frames:][9,:,33:48])
#         test_lossesMAE_z_ts10 += test_lossMAE_z_ts10.item() 
#         test_lossMSE_z_ts10 = criterion2(tp[9,:,33:48].to(device), tactile[context_frames:][9,:,33:48])
#         test_lossesMSE_z_ts10 += test_lossMSE_z_ts10.item()

# performance_data = []
# performance_data.append(["test loss MAE(L1): ", (test_lossesMAE / index__)])
# performance_data.append(["test loss MSE: ", (test_lossesMSE / index__)])
# performance_data.append(["test loss MAE(L1) timestep 1: ", (test_lossesMAE_x / index__)])
# performance_data.append(["test loss MSE timestep 1: ", (test_lossesMSE_x / index__)])
# performance_data.append(["test loss MAE(L1) timestep 5: ", (test_lossesMAE_y / index__)])
# performance_data.append(["test loss MSE timestep 5: ", (test_lossesMSE_y / index__)])
# performance_data.append(["test loss MAE(L1) timestep 10: ", (test_lossesMAE_z / index__)])
# performance_data.append(["test loss MSE timestep 10: ", (test_lossesMSE_z / index__)])
# performance_data.append(["sheer x test loss MAE(L1): ", (test_lossesMAE_x / index__)])
# performance_data.append(["sheer x test loss MSE: ", (test_lossesMSE_x / index__)])
# performance_data.append(["sheer y test loss MAE(L1): ", (test_lossesMAE_y / index__)])
# performance_data.append(["sheer y test loss MSE: ", (test_lossesMSE_y / index__)])
# performance_data.append(["z test loss MAE(L1): ", (test_lossesMAE_z / index__)])
# performance_data.append(["z test loss MSE: ", (test_lossesMSE_z / index__)])
# performance_data.append(["sheer x test loss MAE(L1) timestep 1: ", (test_lossesMAE_x_ts1 / index__)])
# performance_data.append(["sheer x test loss MSE timestep 1: ", (test_lossesMSE_x_ts1 / index__)])
# performance_data.append(["sheer y test loss MAE(L1) timestep 1: ", (test_lossesMAE_y_ts1 / index__)])
# performance_data.append(["sheer y test loss MSE timestep 1: ", (test_lossesMSE_y_ts1 / index__)])
# performance_data.append(["z test loss MAE(L1) timestep 1: ", (test_lossesMAE_z_ts1 / index__)])
# performance_data.append(["z test loss MSE timestep 1: ", (test_lossesMSE_z_ts1 / index__)])
# performance_data.append(["sheer x test loss MAE(L1) timestep 5: ", (test_lossesMAE_x_ts5 / index__)])
# performance_data.append(["sheer x test loss MSE timestep 5: ", (test_lossesMSE_x_ts5 / index__)])
# performance_data.append(["sheer y test loss MAE(L1) timestep 5: ", (test_lossesMAE_y_ts5 / index__)])
# performance_data.append(["sheer y test loss MSE timestep 5: ", (test_lossesMSE_y_ts5 / index__)])
# performance_data.append(["z test loss MAE(L1) timestep 5: ", (test_lossesMAE_z_ts5 / index__)])
# performance_data.append(["z test loss MSE timestep 5: ", (test_lossesMSE_z_ts5 / index__)])
# performance_data.append(["sheer x test loss MAE(L1) timestep 10: ", (test_lossesMAE_x_ts10 / index__)])
# performance_data.append(["sheer x test loss MSE timestep 10: ", (test_lossesMSE_x_ts10 / index__)])
# performance_data.append(["sheer y test loss MAE(L1) timestep 10: ", (test_lossesMAE_y_ts10 / index__)])
# performance_data.append(["sheer y test loss MSE timestep 10: ", (test_lossesMSE_y_ts10 / index__)])
# performance_data.append(["z test loss MAE(L1) timestep 10: ", (test_lossesMAE_z_ts10 / index__)])
# performance_data.append(["z test loss MSE timestep 10: ", (test_lossesMSE_z_ts10 / index__)])
# [print(i) for i in performance_data]

# np.save(model_path + 'pp_performance_data_conv1_model', np.asarray(performance_data))
# print("done")

In [ ]:
print(tactile_predictions[0].shape)
print(tactile_groundtruth[0].shape)

In [ ]:
import matplotlib
import matplotlib.animation as animation
import mpl_toolkits.mplot3d.axes3d as p3
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
import mpl_toolkits.mplot3d.art3d as art3d
from scipy.ndimage.interpolation import map_coordinates

# calculate tactile values for full sample:
time_step_to_test_t1 = 0    # [batch_set, prediction frames(t1->tx)(6), batch_size, features(48)]
time_step_to_test_t5 = 4
time_step_to_test_t10 = 9
predicted_data_t1 = []
predicted_data_t5 = []
predicted_data_t10 = []
groundtruth_data = []
experiment_to_test = 210
for index, batch_set in enumerate(tactile_predictions):
    for batch in range(0, len(batch_set[0])):
        experiment = experiment_time_steps[index][0][batch][0]
#         print(experiment)
        if experiment == experiment_to_test:
            prediction_values = batch_set[time_step_to_test_t1][batch]
            predicted_data_t1.append(prediction_values)
            prediction_values = batch_set[time_step_to_test_t5][batch]
            predicted_data_t5.append(prediction_values)
            prediction_values = batch_set[time_step_to_test_t10][batch]
            predicted_data_t10.append(prediction_values)
            gt_values = tactile_groundtruth[index][time_step_to_test_t1][batch]
            groundtruth_data.append(gt_values)

class image_player():
    def __init__(self, images, save_name, feature):
        self.feature = feature
        self.save_name = save_name
        print(model_path + str(experiment_to_test) + '/' + self.save_name + '_feature_' + str(self.feature) + '.gif')
        self.run_the_tape(images)

    def grab_frame(self):
#         print(self.indexyyy,  end="\r")
#         print(self.images[0].shape)
        frame = self.images[self.indexyyy].permute(1,2,0).cpu().detach().numpy()[:,:,self.feature]*255
        return frame

    def update(self, i):
        plt.title(i)
        self.im1.set_data(self.grab_frame())
        self.indexyyy+=1
        if self.indexyyy == len(self.images):
            self.indexyyy = 0

    def run_the_tape(self, images):
        self.indexyyy = 0
        self.images = images
        ax1 = plt.subplot(1,2,1)
        self.im1 = ax1.imshow(self.grab_frame(), cmap='gray', vmin=0, vmax=255)
        ani = FuncAnimation(plt.gcf(), self.update, interval=20.8, save_count=len(images), repeat=False)
        ani.save(model_path + str(experiment_to_test) + '/' + self.save_name + '_feature_' + str(self.feature) + '.gif')
        plt.show()

# image_player(predicted_data_t1, "pp_predicted_data_t1", 0)
# print('1')
# image_player(predicted_data_t1, "pp_predicted_data_t1", 1)
# print('2')
# image_player(predicted_data_t1, "pp_predicted_data_t1", 2)
# print('3')
# image_player(predicted_data_t5, "pp_predicted_data_t5", 0)
# print('4')
# image_player(predicted_data_t5, "pp_predicted_data_t5", 1)
# print('5')
# image_player(predicted_data_t5, "pp_predicted_data_t5", 2)
# print('6')
# image_player(predicted_data_t10, "pp_predicted_data_t10", 0)
# print('7')
# image_player(predicted_data_t10, "pp_predicted_data_t10", 1)
# print('8')
# image_player(predicted_data_t10, "pp_predicted_data_t10", 2)
# print('9')
# image_player(groundtruth_data, "pp_groundtruth_data", 0)
# print('1')
# image_player(groundtruth_data, "pp_groundtruth_data", 1)
# print('2')
# image_player(groundtruth_data, "pp_groundtruth_data", 2)
# print('3')

# #save numpy arrays for later analysis:
# np_to_save = []
# np_predicted_data_t1 = []
# np_predicted_data_t5 = []
# np_predicted_data_t10 = []
# np_groundtruth_data = []
# for i in range(0, len(predicted_data_t1)):
#     np_predicted_data_t1.append(predicted_data_t1[i].permute(1,2,0).cpu().detach().numpy())
#     np_predicted_data_t5.append(predicted_data_t5[i].permute(1,2,0).cpu().detach().numpy())
#     np_predicted_data_t10.append(predicted_data_t10[i].permute(1,2,0).cpu().detach().numpy())
#     np_groundtruth_data.append(groundtruth_data[i].permute(1,2,0).cpu().detach().numpy())
# np.save(model_path + str(experiment_to_test) + '/pp_np_predicted_data_t1_conv1_model', np.asarray(np_predicted_data_t1))
# np.save(model_path + str(experiment_to_test) + '/pp_np_predicted_data_t5_conv1_model', np.asarray(np_predicted_data_t5))
# np.save(model_path + str(experiment_to_test) + '/pp_np_predicted_data_t10_conv1_model', np.asarray(np_predicted_data_t10))
# np.save(model_path + str(experiment_to_test) + '/pp_np_groundtruth_data_conv1_model', np.asarray(np_groundtruth_data))

In [ ]:
class image_player():
    def __init__(self, images, save_name, feature):
        self.feature = feature
        self.save_name = save_name
        print(model_path + str(experiment_to_test) + '/' + self.save_name + '_feature_' + str(self.feature) + '.gif')
        self.run_the_tape(images)

    def grab_frame(self):
#         print(self.indexyyy,  end="\r")
#         print(self.images[0].shape)
        frame = self.images[self.indexyyy].permute(1,2,0).cpu().detach().numpy()[:,:,self.feature]*255
        return frame

    def update(self, i):
        plt.title(i)
        self.im1.set_data(self.grab_frame())
        self.indexyyy+=1
        if self.indexyyy == len(self.images):
            self.indexyyy = 0

    def run_the_tape(self, images):
        self.indexyyy = 0
        self.images = images
        ax1 = plt.subplot(1,2,1)
        self.im1 = ax1.imshow(self.grab_frame(), cmap='gray', vmin=0, vmax=255)
        ani = FuncAnimation(plt.gcf(), self.update, interval=20.8, save_count=len(images), repeat=False)
        ani.save(model_path + str(experiment_to_test) + '/' + self.save_name + '_feature_' + str(self.feature) + '.gif')
        plt.show()

images = [abs(predicted_data_t10[i] - groundtruth_data[i]) for i in range(len(groundtruth_data))]
image_player(images, "pp_predicted_data_t10_difference", 2)
# image_player(groundtruth_data, "pp_groundtruth_data", 2)

# #save numpy arrays for later analysis:
# np_to_save = []
# np_predicted_data_t1 = []
# np_predicted_data_t5 = []
# np_predicted_data_t10 = []
# np_groundtruth_data = []
# for i in range(0, len(predicted_data_t1)):
#     np_predicted_data_t1.append(predicted_data_t1[i].permute(1,2,0).cpu().detach().numpy())
#     np_predicted_data_t5.append(predicted_data_t5[i].permute(1,2,0).cpu().detach().numpy())
#     np_predicted_data_t10.append(predicted_data_t10[i].permute(1,2,0).cpu().detach().numpy())
#     np_groundtruth_data.append(groundtruth_data[i].permute(1,2,0).cpu().detach().numpy())
# np.save(model_path + str(experiment_to_test) + '/pp_np_predicted_data_t1_conv1_model', np.asarray(np_predicted_data_t1))
# np.save(model_path + str(experiment_to_test) + '/pp_np_predicted_data_t5_conv1_model', np.asarray(np_predicted_data_t5))
# np.save(model_path + str(experiment_to_test) + '/pp_np_predicted_data_t10_conv1_model', np.asarray(np_predicted_data_t10))
# np.save(model_path + str(experiment_to_test) + '/pp_np_groundtruth_data_conv1_model', np.asarray(np_groundtruth_data))

In [ ]:
import cv2
index = 0
prediction_210 = []
ground_truth_210 = []
for batch in range(0, len(tactile_predictions)):
    for batch_no in range(0, len(tactile_predictions[batch][0])):
        if experiments[batch][batch_no][9] == 208:
            index += 1
            reshape_image_gt = cv2.resize(tactile_groundtruth[batch][0][batch_no].permute(1,2,0).cpu().detach().numpy(), dsize=(4, 4), interpolation=cv2.INTER_CUBIC)
            reshape_image_pp = cv2.resize(tactile_predictions[batch][-1][batch_no].permute(1,2,0).cpu().detach().numpy(), dsize=(4, 4), interpolation=cv2.INTER_CUBIC)
            ground_truth_210.append(reshape_image_gt)
            prediction_210.append(reshape_image_pp)

start = 0
end   = len(prediction_210)
plt.plot([i for i in range(start, end)], [ground_truth_210[i][0][0][2] for i in range(start, end)], 'k')
plt.plot([i for i in range(start, end)], [prediction_210[i][0][0][2] for i in range(start, end)], 'r')
plt.ylim((0,1))
plt.show()

In [ ]:
batches = 0
experi = 0 # not time_step = 1
batch_no = 0
time_step = 9
print(experiment_time_steps[batches][experi][batch_no][time_step].shape)